# Machine Translation

English-German Translation

# 1)- Importing key modules

In [0]:
#support both Python 2 and Python 3 with minimal overhead.
from __future__ import absolute_import, division, print_function
#ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [0]:
import string 
from string import digits
import re 
from sklearn.utils import shuffle
from numpy import array, argmax, random, take 
import pandas as pd 
import matplotlib.pyplot as plt 
% matplotlib inline 
pd.set_option('display.max_colwidth', 200)

In [0]:
from keras.models import Model
from keras.models import Sequential 
from keras.layers import Dense, LSTM, Embedding,Input,RepeatVector
from keras.preprocessing.text import Tokenizer
from keras.callbacks import ModelCheckpoint 
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model 
from keras import optimizers 


# 2)- Reading Dataset

In [0]:
lines= pd.read_table('deu.txt', names=['eng', 'deu'])

In [43]:
lines.shape

(195847, 2)

In [0]:
# Lowercase all characters
lines.eng=lines.eng.apply(lambda x: x.lower())
lines.deu=lines.deu.apply(lambda x: x.lower())

In [0]:
# Remove quotes
lines.eng=lines.eng.apply(lambda x: re.sub("'", '', x))
lines.deu=lines.deu.apply(lambda x: re.sub("'", '', x))

In [0]:
exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
lines.eng=lines.eng.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines.deu=lines.deu.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [0]:
# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
lines.eng=lines.eng.apply(lambda x: x.translate(remove_digits))
lines.deu=lines.deu.apply(lambda x: x.translate(remove_digits))

In [0]:
# Remove extra spaces
lines.eng=lines.eng.apply(lambda x: x.strip())
lines.deu=lines.deu.apply(lambda x: x.strip())
lines.eng=lines.eng.apply(lambda x: re.sub(" +", " ", x))
lines.deu=lines.deu.apply(lambda x: re.sub(" +", " ", x))

In [0]:
# Add start and end tokens to target sequences. I am not German so , I am doing so for my ease
lines.deu = lines.mar.apply(lambda x : 'START_ '+ x + ' _END')

In [51]:
lines.sample(10)

,eng,deu
100893,tom treats me like a princess,START_ tom behandelt mich wie eine prinzessin. _END
82164,he is the taller of the two,START_ er ist der größere der beiden. _END
12155,tom sounded mad,START_ tom klang verrückt. _END
81198,a car passed by in the dark,START_ eine auto fuhr in der dunkelheit vorbei. _END
51534,tom went down the hill,START_ tom ging den hügel hinunter. _END
114086,tom switched off all the lights,START_ tom schaltete alle lichter aus. _END
149554,theyre thinking about getting married,START_ sie denken über heirat nach. _END
119376,think carefully before answering,"START_ denk sorgfältig nach, bevor du antwortest! _END"
127425,youre not going to understand it,START_ du wirst es nicht verstehen. _END
8656,tom was biased,START_ tom war voreingenommen. _END


In [0]:
# Vocabulary of English
all_eng_words=set()
for eng in lines.eng:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)
# Vocabulary of German 
all_german_words=set()
for ger in lines.mar:
    for word in ger.split():
        if word not in all_german_words:
            all_german_words.add(word)

In [55]:
# Max Length of source sequence
import numpy as np
lenght_list=[]
for l in lines.eng:
    lenght_list.append(len(l.split(' ')))
max_length_src = np.max(lenght_list)
max_length_src

101

In [56]:
# Max Length of target sequence
lenght_list=[]
for l in lines.deu:
    lenght_list.append(len(l.split(' ')))
max_length_tar = np.max(lenght_list)
max_length_tar

77

In [57]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_german_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_german_words)
num_encoder_tokens, num_decoder_tokens

(15804, 54936)

In [58]:
num_decoder_tokens += 1 # For zero padding
num_decoder_tokens

54937

In [0]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [0]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [61]:
lines = shuffle(lines)
lines.head(10)

,eng,deu
77897,take the road on the right,START_ nehmen sie die rechte straße. _END
48972,its a book on animals,START_ das ist ein buch über tiere. _END
146603,would you help me carry this upstairs,"START_ würdest du mir helfen, dies nach oben zu tragen? _END"
55066,i wonder why he is late,"START_ ich frage mich, wieso er zu spät ist. _END"
13346,dont come again,START_ kommt nicht noch mal. _END
158678,he bought the picture for next to nothing,START_ er kaufte das bild zu einem spottpreis. _END
160431,theres a fine line between love and hate,START_ liebe und hass wohnen dicht beieinander. _END
178727,it seems that tom only eats fruits and vegetables,START_ tom isst anscheinend nur obst und gemüse. _END
53713,he put the luggage down,START_ er stellte das gepäck ab. _END
134449,id like to invite you to the party,START_ ich möchte sie zur party einladen. _END
